# 1. Domača Naloga

## Naloga 1: Linearna regresija

1.a: Dopolni funkcijo `najdi_koeficiente`, ki vrne optimalne vrednosti koeficientov modela linearne regresije. Pomagaj si s prosojnicami iz predavanj, funkcijo za izračun inverza matrike `numpy.linalg.inv(array)` in operatorjem matričnega množenja, `numpy.matmul(array1, array2)` ali `@`. Pravilnost dopolnjene funkcije lahko preveriš s spodnjo kodo. 

Kakšne optimalne vrednosti koeficientov si pričakoval/a in kakšne si dobil/a? 

In [1]:
import numpy as np

In [2]:
def najdi_koeficiente(x_train: np.array, y_train: np.array):
    # Izračunaj optimalne vrednosit koeficientov in jih shrani v spremenljivko coefs
    coefs = np.linalg.inv(x_train.T @ x_train) @ x_train.T @ y_train
    return coefs

In [3]:
# Vzorčimo naključna števila v matriko dimenzij 1000x4,
# ki podaja vrednosti štirih napovednih spremenljivk
x = np.random.random((1000, 4))
# Izračunamo vrednosti ciljne spremenljivke z modelom y = x1 + 5 x2
y = x[:, 0] + 5*x[:, 1]
# Izračunamo optimalne vrednosti koeficientov linearnega modela
coefs = najdi_koeficiente(x, y)

# Preverimo, če je matrika coefs ustreznih dimenzij
assert coefs.shape == (4,)
# Izpišemo vrednosti koeficientov
print(coefs)

[ 1.00000000e+00  5.00000000e+00 -4.00027234e-15 -7.21644966e-16]


1.b Smiselno dopolni funkciji `najdi_koeficiente2` in `napovej` tako, da boš z njima lahko sestavil model, ki bo imel RMSE $< 10^{-10}$ na testnih podatkih, dobljenih z modelom $y = x_1 + 5x_2 + 12$. Pomagaš si lahko s funkcijo `numpy.concatnate(seznam stolpcev, axis=1)`.

In [32]:
from sklearn.metrics import mean_squared_error

In [4]:
def najdi_koeficiente2(x_train, y_train):
    # Izračunaj optimalne vrednosti koeficientov in jih shrani v spremenljivko coefs
    x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
    coefs = np.linalg.inv(x_train.T @ x_train) @ x_train.T @ y_train
    return coefs

In [34]:
def napovej(coefs, x_test):
    # Izračunaj napovedi linarnega modela z vrednostmi koeficientov coefs
    # za primere iz x_test in jih shrani v y_pred
    y_pred = x_test @ coefs[1:] + coefs[0]
    return y_pred

In [61]:
# Vzorčimo matriko naključnih števil velikosti 1000x4
x = np.random.random((1000, 4))
# Definiramo ciljno spremenljivko
y = x[:, 0] + 5*x[:, 1] + 12
# Najdemo koeficiente
coefs = najdi_koeficiente2(x, y)

# Izpišemo koeficiente, funkcijo "{nekej}".join(seznam nizov) pretvori seznam nizov v niz oblike: "{seznam[0]}{nekej}{seznam[1]}{nekej}...{nekej}{seznam[-1]}"
# Funkcija np.round(stevilo, celo število) zaokrozi število na "celo število" decimalnih mest
print(f"Coeficients: {', '.join([str(np.round(c, 3)) for c in coefs])}")

# Vzorčimo matriko naključnih števil velikosti 1000x4 za testno množico
x_test = np.random.random((1000, 4))
# Izračunamo ciljno spremenljivko
y_test = x_test[:, 0] + 5*x_test[:, 1] + 12
# Napovemo cilne vrednosti za testne napovedne podatke 
y_pred = napovej(coefs, x_test)
# Izračunamo RMSE testnih napovedi
error = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {error}")
# Preverimo, da je napaka manjša od 10^{-10}
assert error < 1e-10

Coeficients: 12.0, 1.0, 5.0, -0.0, 0.0
RMSE: 1.4402628516812352e-14


1.c: Naloži podatkovno množico iz datoteke `dn1_1.npy` in jo razdeli na podatke o napovednih spremenljivkah $x$ in podatke o ciljni spremenljivki $y$. Ciljna spremenljivka je v zadnjem stolpcu datoteke s podatkovno množico.

In [6]:
d1 = np.load('dn1_1.npy')
x_train_1 = d1[:,:-1]
y_train_1 = d1[:, -1].reshape(-1, 1)

2000

1.d: Čim bolje oceni točnost (ali napako) napovednega modela iz naloge 1.b. Predpostavi, da bo model naučen na vseh podatkih iz naloge 1.c in da bodo novi podatki prihajali iz iste domene (in verjetnostne porazdelitve) kot učni podatki.

## Naloga 2: Logistična regresija

2.a: Preberi podatkovno množico z diskretno ciljno spremenljivko iz datoteke `dn1_2.npz` (pomagaš si lahko z nalogo 2 iz vaj 3). Izračunaj nekaj preprostih statistik za vsako spremenljivko (stolpec) in podatke po potrebi predprocesiraj za potrebe učenja linearnega modela.

In [9]:
d2 = np.load('dn1_2.npz')
X = d2['x']
y = d2['y']

# Povprecna vrednost spremenljivk
X_mean = np.mean(X, axis=0)
print("Mean: ", X_mean)

# Standardni odklon spremenljivk
X_std = np.std(X, axis=0)
print("Standard deviation: ", X_std)

# Kvartili spremenljivk 
X_q = np.quantile(X, [0.25, 0.5, 0.75], axis=0)
print("Quantiles: ", X_q)

# Normaliziraj podatke
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

Mean:  [-0.01664422  3.93709115  5.49976384  0.98036683  1.48634641  0.59712199
  0.74531457  0.50850014]
Standard deviation:  [1.76899549 2.90060088 0.85557046 0.56830939 1.44224079 0.23137687
 0.14472136 0.28875373]
Quantiles:  [[-1.55322015  1.39949574  4.76044785  0.49233866  0.22268825  0.39406743
   0.6122823   0.25612893]
 [-0.06375526  3.84917229  5.48381429  0.96489185  1.47748935  0.5900379
   0.74912656  0.51148163]
 [ 1.5777605   6.45473438  6.24392828  1.46454515  2.74878036  0.79782948
   0.86835291  0.75839118]]


2.b: Ovrednoti točnost (napako) in stabilnost modela logistične regresije. Poskrbi, da bodo eksperimenti ponovljivi in točnost modela čim bolje ocenjena. Če uporabiš prečno preverjanje, naj število vzorcev ne presega 10.

In [16]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

np.random.seed(42)

logreg = LogisticRegression()
skf = StratifiedKFold(n_splits=5, shuffle=True)


for i, (train_index, test_index) in enumerate(skf.split(X_norm, y)):
    # Razdeli na učno in testno množico
    x_train, x_test = X_norm[train_index], X_norm[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Učenje modela
    logreg.fit(x_train, y_train)

    # Napovedovanje in ocenjevanje
    acc = accuracy_score(y_test, logreg.predict(x_test))
    print("Accuracy score:", acc)

Accuracy score: 0.9675
Accuracy score: 0.9725
Accuracy score: 0.96
Accuracy score: 0.96
Accuracy score: 0.945


2.c: Domenski ekspert vam je namignil, da je vrednost ciljne spremenljivke korelirana s kvadratom prve napovedne spremenljivke (X[:, 0]) in produktom prvih treh napovednih sprememnljivk (X[:, 0], X[:, 1], X[:, 2]). Ali lahko namig uporabiš za izboljšanje modela logistične regresije? Če ga lahko, ovrednoti izboljšanje.

Opomba: Opozorilu o problemu s konvergenco se lahko izognete tako, da povečate število iteracij logistične regresije (npr. na 1000).

In [19]:
# Dodajanje novih spremenljivk 
new_features = np.column_stack((X[:, 0]**2, X[:, 0]*X[:, 1]*X[:, 2]))
X_new = np.hstack((X, new_features))

# Normaliziraj podatke
X_new_norm = (X_new - np.mean(X_new, axis=0)) / np.std(X_new, axis=0)

for i, (train_index, test_index) in enumerate(skf.split(X_new_norm, y)):
    # Razdeli na učno in testno množico
    x_train, x_test = X_norm[train_index], X_norm[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Učenje modela
    logreg.fit(x_train, y_train)

    # Napovedovanje in ocenjevanje
    acc = accuracy_score(y_test, logreg.predict(x_test))
    print("Accuracy score:", acc)

Accuracy score: 0.96
Accuracy score: 0.9625
Accuracy score: 0.965
Accuracy score: 0.935
Accuracy score: 0.97


2.d: Ali lahko s pomočjo logistične regresije ugotoviš nabor napovednih spremenljivk, ki za klasifikacijo niso pomembne? Če lahko ugotoviš, potem poročaj katere spremenljivke so v tem naboru, zakaj sodiš, da so v njem, in kakšna je točnost modela logistične regresije, če jih odstraniš iz podatkovne množice? 

Ali se rezultati med prejšnjim in sedanjim modelom zelo razlikujejo? Kaj je po tvoje razlog za to?

## Naloga 3: K-najbližjih sosedov

3.a: Preberi podatkovno množico z diskretno ciljno spremenljiko iz datoteke `dn1_3.npz`.  

In [73]:
d3 = np.load('dn1_3.npz')

3.b: Sestavi napovedni model k-najbližjih sosedov s čim nižjim RMSE-jem. Poskrbi, da poleg ocene točnosti modela, poročaš tudi stabilnost ocene. Opiši, kakšen je tvoj model in kako/zakaj si se odločil za vsako izbiro. Opiši tudi eksperimente, ki si jih poskusil/a, a niso izboljšali rezultata.

Opomba: Pomen spremenljivk je sledeč:
- $x_1$: Zaporedna številka diamanta v bazi
- $x_2$: Število karatov
- $x_3$: Procent globine diamanta ($\frac{2\cdot z}{x+y}$)
- $x_4$: Razmerje med širino vrha in najširšo točko
- $x_5$: Dolžina diamanta
- $x_6$: Širina diamanta
- $x_7$: Globina diamanta
- $x_8$: Oddaljenost najdišča od ekvatorja
- $x_9$: Kvaliteta brusa ("Ideal": 4, "Premium": 3, "Very Good": 2, "Good": 1, "Fair": 0)
- $x_{10}$: Barva ("D": 0, "E": 1, "F": 2, "G": 3, "H": 4, "I": 5, "J":6)
- $x_{11}$: Čistost ("I1": 0, "SI2": 1, "SI1": 2, "VS2": 3, "VS1": 4, "VVS2": 5, "VVS1": 6, "IF": 7)
- $y$: Cena diamanta


In [ ]:
# Logika, normalizacija, dummy varables, dolocitev k-ja